In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as spy
import seaborn as sns
import sklearn.preprocessing as preprocessing
from IPython.display import display
from itertools import combinations
from sklearn.decomposition import PCA
from statsmodels.stats.proportion import proportions_ztest

# Change output defaults.
np.set_printoptions(precision=3, suppress=True)
pd.set_option('display.precision', 3)
pd.set_option('display.chop_threshold', 0.001)
sns.set_style('whitegrid')
sns.set_context('poster')

# Universe of cryptocurrencies we focus on.
CRYPTO_FILES = [('ripple_price', 'rip'), ('bitcoin_price', 'btc'),
                ('ethereum_price', 'eth'), ('litecoin_price', 'ltc'),
                ('monero_price', 'mon'), ('nem_price', 'nem'),
                ('dash_price', 'dash')]

In [25]:
def load_returns_matrix (files, tdelta=pd.Timedelta(days=30), center=True, 
                         scale=True):
    """Returns cryptocurrency rolling returns in three formats.
    
    Args:
        files (list of tuples): (asset name, file path) tuples. Asset name 
            used in column headers for DF.
        tdelta (Timedelta, default 30 days): Time period over which to compute 
            rolling returns.
    
    Returns:
        dfout (DataFrame): Returns without centering/scaling.
        dfout_adj (DataFrame): Returns with centering/scaling (depending 
            on input vals for center and scale).
        xout (np.matrix): Returns matrix with centering/scaling (depending 
            on args), i.e., without index or column names.
    """
    dfs = []
    for file, name in files:
        path = 'cryptocurrencypricehistory//{}.csv'.format(file)
        df = pd.read_csv(path, usecols=['Date', 'Close'])
        df['Date'] = pd.to_datetime(df['Date'])
        df = df[['Date', 'Close']]
        df.set_index('Date', drop=True, inplace=True)
        df.rename(columns={'Close':name}, inplace=True)
        dfs.append(df)
    dfout = pd.concat(dfs, axis=1, join='inner')
    dfout = dfout.pct_change(periods=1, freq=tdelta)
    dfout.dropna(axis=0, how='any', inplace=True)
    xout = preprocessing.scale(dfout, axis=0, with_mean=center, with_std=scale)
    dfout_adj = pd.DataFrame(xout, columns=dfout.columns, index=dfout.index)
    return dfout, dfout_adj, np.matrix(xout)


def create_proportion_of_variation_df (eigvals):
    total_var = sum(eigvals)
    cols = ['component', 'eigenvalue', 'proportion', 'cumulative']
    df = pd.DataFrame(columns=cols)
    cum = 0
    for i, e in enumerate(eigvals):
        cum += e
        row = {'component':i + 1,
               'eigenvalue':e,
               'proportion':e/total_var,
               'cumulative':cum/total_var}
        df = df.append(row, ignore_index=True)
    df['component'] = df['component'].astype(int)
    df.set_index('component', drop=True, inplace=True)
    return df


def create_eigvec_df (eigvecs):
    """Create DataFrame of eigenvectors."""
    idx = [i + 1 for i in range(len(eigvecs))]
    df = pd.DataFrame(index=idx)
    for i, v in enumerate(eigvecs):
        df['V{}'.format(i + 1)] = v
    return df

In [26]:
# Init data and obtain covariance matrix.
df_returns_unadj, df_returns, X = load_returns_matrix(CRYPTO_FILES,
                                                      center=True, scale=True)
n, p = X.shape
C = np.cov(X, rowvar=False)  # covariance matrix
assert n == len(df_returns.index)
print('Number of returns in data: {}'.format(n))
df_returns.head()

2015-08-07 00:00:00
2017-11-07 00:00:00
Number of returns in data: 794


,rip,btc,eth,ltc,mon,nem,dash
Date,,,,,,,
2015-09-06,-0.291,-1.261,-1.125,-0.946,-0.783,-0.776,-0.828
2015-09-07,-0.314,-0.998,0.299,-0.803,-0.717,-0.739,-0.814
2015-09-08,-0.345,-0.998,0.427,-0.825,-0.684,-0.693,-0.852
2015-09-09,-0.353,-1.077,0.357,-0.915,-0.680,-0.772,-0.782
2015-09-10,-0.343,-1.157,-0.378,-0.979,-0.619,-0.766,-0.845


### Basic features of unstandardized price return data

In [4]:
print('Price return statistics without centering/scaling:')
print('Mean returns:')
display(df_returns_unadj.mean(axis=0))
print('S.D. of returns:')
display(df_returns_unadj.std(axis=0))
print('Cumulative return:')
display((df_returns_unadj + 1).prod(axis=0))
print('Correlation:')
df_returns.corr()

Price return statistics without centering/scaling:
Mean returns:


rip     0.395
btc     0.150
eth     0.410
ltc     0.173
mon     0.364
nem     0.585
dash    0.277
dtype: float64

S.D. of returns:


rip     1.468
btc     0.232
eth     0.836
ltc     0.475
mon     0.873
nem     1.190
dash    0.620
dtype: float64

Cumulative return:


rip     1.841e+43
btc     8.094e+41
eth     2.544e+71
ltc     7.702e+36
mon     3.682e+66
nem     1.561e+98
dash    3.883e+60
dtype: float64

Correlation:


,rip,btc,eth,ltc,mon,nem,dash
rip,1.000,0.305,0.325,0.694,0.066,0.630,0.037
btc,0.305,1.000,0.164,0.322,0.102,0.349,0.031
eth,0.325,0.164,1.000,0.192,0.171,0.361,0.470
ltc,0.694,0.322,0.192,1.000,0.044,0.428,0.018
mon,0.066,0.102,0.171,0.044,1.000,0.114,0.128
nem,0.630,0.349,0.361,0.428,0.114,1.000,0.133
dash,0.037,0.031,0.470,0.018,0.128,0.133,1.000


In [5]:
# Get SVD breakdown, cast into matrices.
U, s, V = np.linalg.svd(X)
U = np.matrix(U)  # n x n matrix
S = np.zeros((n, p)) 
S[:p, :p] = np.diag(s)
S = np.matrix(S)  # n x p matrix
V = np.matrix(V).T  # p x p matrix
eigvecs = [np.ravel(V[:,i]) for i in range(p)]

# Get eigenvalues through the singular values.
eigvals = [(s_**2)/(n - 1) for s_ in s]

In [6]:
# Reconstruct data matrix X and covariance matrix C using SVD properties.
# Check equality by looking at the norm of their difference.
Xreconstr = U*S*V.T
print('Norm[Xreconstr - X] = {:.2f}'.format(np.linalg.norm(Xreconstr - X)))

Creconstr = V*((S.T*S)/(n - 1))*V.T
print('Norm[Creconstr - C] = {:.2f}'.format(np.linalg.norm(Creconstr - C)))

# Get principal components (n-by-p matrix).
XV = X*V

Norm[Xreconstr - X] = 0.00
Norm[Creconstr - C] = 0.00


**Inner product should be 0 between all eigenvectors and principal components since they are orthogonal. Test this out for the first two eigenvectors and PCs.**

In [7]:
pc1, pc2 = np.ravel(XV[:, 0]), np.ravel(XV[:, 1])
v1, v2 = np.ravel(V[:, 0]), np.ravel(V[:, 1])
print('Inner product between 1st and 2nd eigenvectors: {:.2f}'.format(
      np.inner(v1, v2)))
print('Inner product between 1st and 2nd PCs: {:.2f}'.format(
      np.inner(pc1, pc2)))

Inner product between 1st and 2nd eigenvectors: 0.00
Inner product between 1st and 2nd PCs: 0.00


In [8]:
# Create tables for proportion of variation and eigenvectors.
df_variation_svd = create_proportion_of_variation_df(eigvals)
df_eigvec_svd = create_eigvec_df(eigvecs)

In [9]:
print('Eigenvectors obtained from SVD')
df_eigvec_svd

Eigenvectors obtained from SVD


,V1,V2,V3,V4,V5,V6,V7
1,0.517,-0.237,0.105,-0.286,0.015,-0.013,-0.764
2,0.334,-0.150,-0.305,0.867,0.095,-0.064,-0.091
3,0.358,0.508,0.204,0.007,-0.228,-0.712,0.118
4,0.455,-0.309,0.065,-0.226,0.611,-0.087,0.511
5,0.137,0.327,-0.880,-0.311,0.056,0.027,-0.012
6,0.487,-0.063,0.037,-0.044,-0.640,0.472,0.351
7,0.174,0.676,0.274,0.130,0.390,0.508,-0.103


In [10]:
# Print eigenvectors obtained using sklearn.
pca = PCA().fit(X)
sk_comp = pca.components_
print('Eigenvectors obtained from sklearn PCA.components_')
sk_comp.T

Eigenvectors obtained from sklearn PCA.components_


array([[ 0.517, -0.237, -0.105, -0.286, -0.015,  0.013,  0.764],
       [ 0.334, -0.15 ,  0.305,  0.867, -0.095,  0.064,  0.091],
       [ 0.358,  0.508, -0.204,  0.007,  0.228,  0.712, -0.118],
       [ 0.455, -0.309, -0.065, -0.226, -0.611,  0.087, -0.511],
       [ 0.137,  0.327,  0.88 , -0.311, -0.056, -0.027,  0.012],
       [ 0.487, -0.063, -0.037, -0.044,  0.64 , -0.472, -0.351],
       [ 0.174,  0.676, -0.274,  0.13 , -0.39 , -0.508,  0.103]])

### Variation Explained using Both Methods

Total variation should be equal to the sum of crypto's variances (7 cryptos * 1 unit variance = 7%).

In [11]:
print('Variation Explained using Eigenvalues from SVD')
df_variation_svd

Variation Explained using Eigenvalues from SVD


,eigenvalue,proportion,cumulative
component,,,
1,2.657,0.379,0.379
2,1.376,0.196,0.575
3,0.941,0.134,0.710
4,0.769,0.110,0.819
5,0.564,0.080,0.900
6,0.464,0.066,0.966
7,0.239,0.034,1.000


In [12]:
idx = [i + 1 for i in range(p)]
data = {'component': idx,
       'eigenvalue': pca.explained_variance_,
       'proportion': pca.explained_variance_ratio_}
df_variation_sk = pd.DataFrame(data)
df_variation_sk.set_index('component', drop=True, inplace=True)

print('Variation Explained using sklearn')
display(df_variation_sk)
print('Total variation: {:.2f}'.format(df_variation_sk['eigenvalue'].sum()))

Variation Explained using sklearn


,eigenvalue,proportion
component,,
1,2.657,0.379
2,1.376,0.196
3,0.941,0.134
4,0.769,0.110
5,0.564,0.080
6,0.464,0.066
7,0.239,0.034


Total variation: 7.01


### Comparing sklearn's PCA and SVD

The principal components retrieved using `sklearn` and SVD should be the same.

The following cell shows this is true for the first and second principal component.

In [13]:
# Get principal components using sklearn.
XReduced_sk = pca.transform(X)
assert XReduced_sk.shape == (n, p)

In [14]:
x1_reduced_sk = np.ravel(XReduced_sk[:,0])
x2_reduced_sk = np.ravel(XReduced_sk[:,1])

x1_reduced_diff = np.linalg.norm(x1_reduced_sk - pc1)
x2_reduced_diff = np.linalg.norm(x2_reduced_sk - pc2)

print("Norm of difference between sklearn's principal component and " \
     "component created using SVD:")
print('PC1: {:.2f}'.format(x1_reduced_diff))
print('PC2: {:.2f}'.format(x2_reduced_diff))

Norm of difference between sklearn's principal component and component created using SVD:
PC1: 0.00
PC2: 0.00


## Loadings

In [15]:
# Calc loadings and put into DataFrame.
loadings = np.matrix(eigvecs * np.sqrt(np.abs(eigvals)))
idx_loading = pd.Series([i + 1 for i in range(p)], name='component')
df_loadings = pd.DataFrame(loadings, columns=df_returns.columns, 
                           index=idx_loading)
display(df_loadings)

,rip,btc,eth,ltc,mon,nem,dash
component,,,,,,,
1,0.844,0.391,0.347,0.399,0.103,0.331,0.085
2,-0.386,-0.176,0.492,-0.271,0.245,-0.043,0.330
3,0.171,-0.358,0.198,0.057,-0.660,0.025,0.134
4,-0.466,1.017,0.007,-0.198,-0.233,-0.030,0.064
5,0.024,0.111,-0.221,0.536,0.042,-0.436,0.191
6,-0.021,-0.075,-0.691,-0.076,0.020,0.321,0.248
7,-1.245,-0.106,0.114,0.448,-0.009,0.239,-0.051


## Can Cryptocurrencies be Viewed as an Asset Class? 

In [16]:
def count_same_sign(df_load, crypto_pair, n_comp):
    """Returns number of times loadings are of the same sign for pair of 
    cryptocurrencies among n_comp components.
    """
    same_sign = 0
    crypt1, crypt2 = crypto_pair
    for i in range(1, n_comp + 1):
        loadings = [df_load.loc[i][crypt1], df_load.loc[i][crypt2]]
        signs = [int(s) for s in np.sign(loadings)]
        if abs(sum(signs)) == 2:
            same_sign += 1
    return same_sign

In [17]:
# Prepare DataFrame storing number of times the loadings for a pair of 
# cryptos have the same signs for a certain number of PCs.

crypto_pairs = list(combinations(df_loadings.columns, 2))
ss_n_components = 5
data_rows = []
for pair in crypto_pairs:
    count = count_same_sign(df_loadings, pair, ss_n_components)
    data_rows.append([pair[0], pair[1], count])

cols = ['Crypto 1', 'Crypto 2', 'Same Sign Loading']
same_sign_df = pd.DataFrame(data_rows, columns=cols)
display(same_sign_df)

,Crypto 1,Crypto 2,Same Sign Loading
0,rip,btc,3
1,rip,eth,2
2,rip,ltc,5
3,rip,mon,3
4,rip,nem,4
5,rip,dash,3
6,btc,eth,2
7,btc,ltc,3
8,btc,mon,3
9,btc,nem,2


### Proportion z-test

In [18]:
# Perform proportion z-test using proportion of times the crypto pair 
# had the same loading signs.

same_sign_counts = same_sign_df['Same Sign Loading'].tolist()
count_greater = len([i for i in same_sign_counts if i > (ss_n_components/2)])
null_hypo_value = 0.5
zstat, pval = proportions_ztest(count_greater, 
                                nobs=len(same_sign_df.index), 
                                value=null_hypo_value,
                                alternative='larger', 
                                prop_var=null_hypo_value)

print('z-stat: {:.3f}\np-value: {:.3%}'.format(zstat, pval))

z-stat: 1.528
p-value: 6.332%


In [19]:
# Perform proportion z-test using total counts that a crypto pair had the 
# same loading signs.

total_same_sign = sum(same_sign_counts)
total_obs = ss_n_components * len(same_sign_df.index)
zstat, pval = proportions_ztest(total_same_sign, 
                                nobs=total_obs, 
                                value=null_hypo_value,
                                alternative='larger', 
                                prop_var=null_hypo_value)

print('z-stat: {:.3f}\np-value: {:.3%}'.format(zstat, pval))

z-stat: 1.659
p-value: 4.855%


### Varimax Rotation on Loadings

In [20]:
def varimax(X, gamma = 1.0, q = 20, tol = 1e-6):
    p,k = X.shape
    R = spy.eye(k)
    d=0
    for i in range(q):
        d_old = d
        Lam = spy.dot(X, R)
        u,s,vh = spy.linalg.svd(spy.dot(X.T,spy.asarray(Lam)**3 - 
                                        (gamma/p) * 
                                        spy.dot(Lam, 
                                                spy.diag(spy.diag(spy.dot(
                                                    Lam.T,Lam))))))
        R = spy.dot(u,vh)
        d = spy.sum(s)
        if d_old!=0 and d/d_old < 1 + tol: break
    return spy.dot(X, R)

In [21]:
# Rotate loadings using Varimax rotation matrix.

loadings_rot = np.matrix(varimax(loadings))
df_loadings_rot = pd.DataFrame(loadings_rot, columns=df_returns.columns, 
                               index=idx_loading)
display(df_loadings_rot.round(3))

,rip,btc,eth,ltc,mon,nem,dash
component,,,,,,,
1,0.430,-0.013,0.070,0.032,-0.006,1.038,-0.061
2,-0.249,-0.020,0.176,-0.137,0.032,-0.144,0.730
3,0.059,-0.196,0.065,-0.039,-0.778,0.010,-0.026
4,-0.172,1.147,0.015,0.000,0.074,-0.011,-0.005
5,-0.054,-0.002,0.000,0.742,0.044,0.067,-0.129
6,0.020,-0.041,-0.778,0.000,0.066,-0.145,-0.147
7,-1.330,0.124,0.009,0.012,0.020,-0.215,0.060


In [22]:
df_returns_unadj.head()

,rip,btc,eth,ltc,mon,nem,dash
Date,,,,,,,
2015-09-06,-0.032,-0.142,-0.531,-0.276,-0.320,-0.338,-0.236
2015-09-07,-0.066,-0.081,0.659,-0.208,-0.261,-0.294,-0.227
2015-09-08,-0.111,-0.081,0.767,-0.218,-0.233,-0.238,-0.251
2015-09-09,-0.123,-0.099,0.708,-0.261,-0.229,-0.333,-0.208
2015-09-10,-0.108,-0.118,0.093,-0.291,-0.176,-0.326,-0.247


In [23]:
min(df_returns_unadj.index)

Timestamp('2015-09-06 00:00:00', freq='D')

In [24]:
max(df_returns_unadj.index)

Timestamp('2017-11-07 00:00:00', freq='D')

In [ ]:
loadings_rot.to_latex()